In [1]:
import wrangle as w
import model as m
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as stats
import nltk
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

imports loaded successfully, awaiting commands...


In [2]:
train = pd.read_parquet('train.parquet')
validate = pd.read_parquet('validate.parquet')
test = pd.read_parquet('test.parquet')

In [ ]:
train.

In [3]:
sm_train1 = int(round(len(train[train.company_response_to_consumer=='Closed with explanation'])*.2,0))
sm_train2 = int(round(len(train[train.company_response_to_consumer=='Closed with non-monetary relief'])*.2,0))
sm_train3 = int(round(len(train[train.company_response_to_consumer=='Closed with monetary relief'])*.2,0))
sm_train4 = int(round(len(train[train.company_response_to_consumer=='Untimely response'])*.2,0))
sm_train5 = int(round(len(train[train.company_response_to_consumer=='Closed'])*.2,0))
sm_val1 = int(round(len(validate[validate.company_response_to_consumer=='Closed with explanation'])*.2,0))
sm_val2 = int(round(len(validate[validate.company_response_to_consumer=='Closed with non-monetary relief'])*.2,0))
sm_val3 = int(round(len(validate[validate.company_response_to_consumer=='Closed with monetary relief'])*.2,0))
sm_val4 = int(round(len(validate[validate.company_response_to_consumer=='Untimely response'])*.2,0))
sm_val5 = int(round(len(validate[validate.company_response_to_consumer=='Closed'])*.2,0))
sm_test1 = int(round(len(test[test.company_response_to_consumer=='Closed with explanation'])*.2,0))
sm_test2 = int(round(len(test[test.company_response_to_consumer=='Closed with non-monetary relief'])*.2,0))
sm_test3 = int(round(len(test[test.company_response_to_consumer=='Closed with monetary relief'])*.2,0))
sm_test4 = int(round(len(test[test.company_response_to_consumer=='Untimely response'])*.2,0))
sm_test5 = int(round(len(test[test.company_response_to_consumer=='Closed'])*.2,0))

small_train1 = train[train.company_response_to_consumer=='Closed with explanation'].sample(sm_train1,random_state=123)
small_train2 = train[train.company_response_to_consumer=='Closed with non-monetary relief'].sample(sm_train2,random_state=123)
small_train3 = train[train.company_response_to_consumer=='Closed with monetary relief'].sample(sm_train3,random_state=123)
small_train4 = train[train.company_response_to_consumer=='Untimely response'].sample(sm_train4,random_state=123)
small_train5 = train[train.company_response_to_consumer=='Closed'].sample(sm_train5,random_state=123)
small_val1 = validate[validate.company_response_to_consumer=='Closed with explanation'].sample(sm_val1,random_state=123)
small_val2 = validate[validate.company_response_to_consumer=='Closed with non-monetary relief'].sample(sm_val2,random_state=123)
small_val3 = validate[validate.company_response_to_consumer=='Closed with monetary relief'].sample(sm_val3,random_state=123)
small_val4 = validate[validate.company_response_to_consumer=='Untimely response'].sample(sm_val4,random_state=123)
small_val5 = validate[validate.company_response_to_consumer=='Closed'].sample(sm_val5,random_state=123)
small_test1 = test[test.company_response_to_consumer=='Closed with explanation'].sample(sm_test1,random_state=123)
small_test2 = test[test.company_response_to_consumer=='Closed with non-monetary relief'].sample(sm_test2,random_state=123)
small_test3 = test[test.company_response_to_consumer=='Closed with monetary relief'].sample(sm_test3,random_state=123)
small_test4 = test[test.company_response_to_consumer=='Untimely response'].sample(sm_test4,random_state=123)
small_test5 = test[test.company_response_to_consumer=='Closed'].sample(sm_test5,random_state=123)

small_train = pd.concat([small_train1,small_train2,small_train3,small_train4,small_train5])
small_val = pd.concat([small_val1,small_val2,small_val3,small_val4,small_val5])
small_test = pd.concat([small_test1,small_test2,small_test3,small_test4,small_test5])

X_train = m.encode(small_train)
X_train = X_train.drop(columns=['date_received','company_response_to_consumer','clean','state','company_name','tags','product_bins'])
y_train = small_train['company_response_to_consumer']
X_val = m.encode(small_val)
X_val = X_val.drop(columns=['date_received','company_response_to_consumer','clean','state','company_name','tags','product_bins'])
y_val = small_val['company_response_to_consumer']
X_test = m.encode(small_test)
X_test = X_test.drop(columns=['date_received','company_response_to_consumer','clean','state','company_name','tags','product_bins'])
y_test = small_test['company_response_to_consumer']
X_train.head()

,lemon,Older American,"Older American, Servicemember",Servicemember,credit_card,credit_report,debt_collection,loans,money_service,mortgage
48045,trying contact grain technology inc debt owed ...,0,0,0,0,0,1,0,0,0
1164880,bought large rug start vacation called filed d...,0,0,0,1,0,0,0,0,0
463976,original complaint went car dealership ohio lo...,0,0,0,0,1,0,0,0,0
490359,synopsis loan demand estate loan originated an...,0,0,0,0,0,0,0,0,1
316427,may concern writing dispute fraudulent charge ...,0,0,0,0,1,0,0,0,0


In [4]:
X_train_tf,X_val_tf,X_test_tf = m.make_tfidf(X_train, X_val, X_test)

In [5]:
encoded_train = X_train.iloc[:,1:]
encoded_val = X_val.iloc[:,1:]
encoded_test = X_test.iloc[:,1:]



X_train_tfe = encoded_train.merge(X_train_tf,left_index=True, right_index=True)
X_val_tfe = encoded_val.merge(X_val_tf,left_index=True, right_index=True)
X_test_tfe = encoded_test.merge(X_test_tf,left_index=True, right_index=True)

label_encoder = LabelEncoder()

y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)

hyperparams = [
    {'max_depth': 5, 'learning_rate': 0.1, 'n_estimators': 15}]

def xgboost_hyperparam_search(X_train, y_train, X_val, y_val, hyperparams):
    results = []
    
    for params in hyperparams:
        model = xgb.XGBClassifier(**params)
        model.fit(X_train, y_train)
        
        train_pred = model.predict(X_train)
        train_acc = accuracy_score(y_train, train_pred)
        
        val_pred = model.predict(X_val)
        val_acc = accuracy_score(y_val, val_pred)
        
        results.append({'Parameters': params, 'Train Accuracy': train_acc, 'Validation Accuracy': val_acc})
    
    df_results = pd.DataFrame(results)
    df_results = df_results.sort_values(by='Validation Accuracy', ascending=False).head(10)
    return df_results

xgboost_hyperparam_search(X_train_cve, y_train_encoded, X_val_cve, y_val_encoded, hyperparams)

xgboost_hyperparam_search(X_train_cve, y_train_encoded, X_val_cve, y_val_encoded, hyperparams)

import lightgbm as lgb
from sklearn.metrics import roc_auc_score

def train_lightgbm_model(x_train, y_train, x_val, y_val, hyperparams):
    # Create LightGBM dataset
    train_data = lgb.Dataset(x_train, label=y_train)
    val_data = lgb.Dataset(x_val, label=y_val)

    # Set hyperparameters
    params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting_type': 'gbdt',
        'num_leaves': hyperparams['num_leaves'],
        'learning_rate': hyperparams['learning_rate'],
        'feature_fraction': hyperparams['feature_fraction'],
        'bagging_fraction': hyperparams['bagging_fraction'],
        'bagging_freq': hyperparams['bagging_freq'],
        'verbose': 0
    }

    # Train the model
    model = lgb.train(params,
                      train_data,
                      valid_sets=[train_data, val_data],
                      num_boost_round=hyperparams['num_boost_round'],
                      early_stopping_rounds=hyperparams['early_stopping_rounds'],
                      verbose_eval=hyperparams['verbose_eval'])

    # Predict on validation data
    y_pred = model.predict(x_val)

    # Calculate AUC score
    auc_score = roc_auc_score(y_val, y_pred)

    return model, auc_score


train_lightgbm_model(x_train, y_train, x_val, y_val, hyperparams)

In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

def run_decision_tree_classifier(X_train, y_train, X_val, y_val, **kwargs):
    # Initialize the decision tree classifier with hyperparameters
    classifier = DecisionTreeClassifier(**kwargs)
    
    # Train the classifier
    classifier.fit(X_train, y_train)
    
    # Make predictions on the validation set
    y_pred = classifier.predict(X_val)
    
    # Calculate the accuracy of the classifier
    accuracy = accuracy_score(y_val, y_pred)
    
    # Create a dataframe to hold the results
    results = pd.DataFrame({
        'Actual': y_val,
        'Predicted': y_pred
    })
    
    # Print the accuracy
    print("Accuracy:", accuracy)
    
    # Return the results dataframe and the trained classifier
    return results, classifier


In [7]:

hyperparameters = {
    'max_depth': 5,
    'min_samples_split': 2
}

results, classifier = run_decision_tree_classifier(X_train_tfe, y_train,X_val_tfe, y_val, **hyperparameters)

print(results)


Accuracy: 0.7905913493352584
                          Actual                Predicted
772702   Closed with explanation  Closed with explanation
641726   Closed with explanation  Closed with explanation
799623   Closed with explanation  Closed with explanation
1081959  Closed with explanation  Closed with explanation
859568   Closed with explanation  Closed with explanation
...                          ...                      ...
671381                    Closed  Closed with explanation
952334                    Closed  Closed with explanation
926664                    Closed  Closed with explanation
899697                    Closed  Closed with explanation
237351                    Closed  Closed with explanation

[49869 rows x 2 columns]


In [8]:
hyperparameters = {
    'max_depth': 10,
    'min_samples_split': 4
}

results, classifier = run_decision_tree_classifier(X_train_tfe, y_train,X_val_tfe, y_val, **hyperparameters)

print(results)

Accuracy: 0.7916942389059336
                          Actual                Predicted
772702   Closed with explanation  Closed with explanation
641726   Closed with explanation  Closed with explanation
799623   Closed with explanation  Closed with explanation
1081959  Closed with explanation  Closed with explanation
859568   Closed with explanation  Closed with explanation
...                          ...                      ...
671381                    Closed  Closed with explanation
952334                    Closed  Closed with explanation
926664                    Closed  Closed with explanation
899697                    Closed  Closed with explanation
237351                    Closed  Closed with explanation

[49869 rows x 2 columns]


In [9]:
hyperparameters = {
    'max_depth': 5,
    'min_samples_split': 10
}

results, classifier = run_decision_tree_classifier(X_train_tfe, y_train,X_val_tfe, y_val, **hyperparameters)

print(results)

Accuracy: 0.7905712967976097
                          Actual                Predicted
772702   Closed with explanation  Closed with explanation
641726   Closed with explanation  Closed with explanation
799623   Closed with explanation  Closed with explanation
1081959  Closed with explanation  Closed with explanation
859568   Closed with explanation  Closed with explanation
...                          ...                      ...
671381                    Closed  Closed with explanation
952334                    Closed  Closed with explanation
926664                    Closed  Closed with explanation
899697                    Closed  Closed with explanation
237351                    Closed  Closed with explanation

[49869 rows x 2 columns]


In [10]:
hyperparameters = {
    'max_depth': 1,
    'min_samples_split': 10
}

results, classifier = run_decision_tree_classifier(X_train_tfe, y_train,X_val_tfe, y_val, **hyperparameters)

print(results)

Accuracy: 0.7879043092903407
                          Actual                Predicted
772702   Closed with explanation  Closed with explanation
641726   Closed with explanation  Closed with explanation
799623   Closed with explanation  Closed with explanation
1081959  Closed with explanation  Closed with explanation
859568   Closed with explanation  Closed with explanation
...                          ...                      ...
671381                    Closed  Closed with explanation
952334                    Closed  Closed with explanation
926664                    Closed  Closed with explanation
899697                    Closed  Closed with explanation
237351                    Closed  Closed with explanation

[49869 rows x 2 columns]


In [11]:
def run_multiple_decision_trees(X_train, y_train, X_val, y_val):
    hyperparameters = list(range(1, 11))  # Range of hyperparameters
    
    results = pd.DataFrame()  # Dataframe to hold the results
    models = []  # List to hold the trained classifiers
    
    for param in hyperparameters:
        print("Running model with hyperparameter:", param)
        
        # Run the decision tree classifier with current hyperparameter
        res, clf, train_acc, val_acc = run_decision_tree_classifier(X_train, y_train, X_val, y_val, max_depth=param)
        
        # Append the results to the dataframe
        results = pd.concat([results, res], ignore_index=True)
        
        # Append the trained classifier to the list
        models.append(clf)
        
    # Sort the models based on validation accuracy
    sorted_models = sorted(zip(models, hyperparameters), key=lambda x: x[0].score(X_val, y_val), reverse=True)
    
    # Print the top 5 models based on validation accuracy
    print("Top 5 models based on validation accuracy:")
    for i, (model, param) in enumerate(sorted_models[:5], 1):
        train_acc = model.score(X_train, y_train)
        val_acc = model.score(X_val, y_val)
        print(f"Model {i}: Hyperparameter: {param}, Train Accuracy: {train_acc:.4f}, Validation Accuracy: {val_acc:.4f}")
    
    # Return the results dataframe and the list of trained classifiers
    return results, models

In [12]:
sentiment_df= w.sentiment_analysis(train)

KeyboardInterrupt: 

In [ ]:
grouped_df = sentiment_df.groupby(['product_bins', 'company_response_to_consumer'])


In [ ]:
result = grouped_df['sentiment'].mean()
result

product_bins     company_response_to_consumer   
bank             Closed                            -0.128198
                 Closed with explanation           -0.066920
                 Closed with monetary relief       -0.059679
                 Closed with non-monetary relief    0.070788
                 Untimely response                 -0.170290
credit_card      Closed                             0.175603
                 Closed with explanation            0.225597
                 Closed with monetary relief        0.166649
                 Closed with non-monetary relief    0.282641
                 Untimely response                  0.146639
credit_report    Closed                             0.189479
                 Closed with explanation            0.128327
                 Closed with monetary relief        0.311037
                 Closed with non-monetary relief    0.088647
                 Untimely response                  0.178200
debt_collection  Closed             

In [ ]:
train.company_response_to_consumer.value_counts()

Closed with explanation            589373
Closed with non-monetary relief    123170
Closed with monetary relief         30578
Untimely response                    2675
Closed                               2245
Name: company_response_to_consumer, dtype: int64

In [ ]:
train.head()

,date_received,company_name,state,tags,company_response_to_consumer,product_bins,clean,lemon,sentiment
944870,2015-05-12,"CITIBANK, N.A.",CA,Average Person,Closed with monetary relief,mortgage,it has been since that i first applied for the...,since first applied refinance today decision c...,-0.9899
570225,2021-07-06,"EQUIFAX, INC.",CA,Average Person,Closed with explanation,credit_report,inquiry ive called and informed them that i di...,inquiry ive called informed applied car inquir...,0.3818
523256,2017-10-11,"Franklin Collection Service, Inc.",GA,Average Person,Closed with explanation,debt_collection,i paid the bill in hopes to get it back but th...,paid bill hope get back said pay different ins...,0.1779
753529,2017-12-18,Credit Plus Inc,CA,Average Person,Closed with explanation,credit_report,unknown inquiry on my credit reports from cred...,unknown inquiry credit report credit plus auth...,0.8555
999772,2016-06-17,Selene Finance LP,OR,Average Person,Closed with explanation,mortgage,selene finance will not help me figure out why...,selene finance help figure many different amou...,-0.1531


## 2nd iteration

In [17]:
from sklearn.metrics import accuracy_score, recall_score

def run_decision_tree_classifier(X_train, y_train, X_val, y_val, **kwargs):
    # Initialize the decision tree classifier with hyperparameters
    classifier = DecisionTreeClassifier(**kwargs)
    
    # Train the classifier
    classifier.fit(X_train, y_train)
    
    # Make predictions on the validation set
    y_pred = classifier.predict(X_val)
    
    # Calculate the accuracy and recall scores of the classifier
    accuracy = accuracy_score(y_val, y_pred)
    recall_train = recall_score(y_train, classifier.predict(X_train), average='micro')
    recall_val = recall_score(y_val, y_pred, average='micro')
    
    # Create a dataframe to hold the results
    results = pd.DataFrame({
        'Actual': y_val,
        'Predicted': y_pred
    })
    
    # Print the accuracy and recall scores
    print("Accuracy:", accuracy)
    print("Recall (Train):", recall_train)
    print("Recall (Validation):", recall_val)
    
    # Return the results dataframe and the trained classifier
    return results, classifier


In [16]:
hyperparameters = {
    'max_depth': 8,
    'min_samples_split': 12
}


run_decision_tree_classifier(X_train_tfe, y_train,X_val_tfe, y_val, **hyperparameters)

Accuracy: 0.7913533457659067
Recall (Train): 0.23937176636949098
Recall (Validation): 0.22815735005161036


(                          Actual                Predicted
 772702   Closed with explanation  Closed with explanation
 641726   Closed with explanation  Closed with explanation
 799623   Closed with explanation  Closed with explanation
 1081959  Closed with explanation  Closed with explanation
 859568   Closed with explanation  Closed with explanation
 ...                          ...                      ...
 671381                    Closed  Closed with explanation
 952334                    Closed  Closed with explanation
 926664                    Closed  Closed with explanation
 899697                    Closed  Closed with explanation
 237351                    Closed  Closed with explanation
 
 [49869 rows x 2 columns],
 DecisionTreeClassifier(max_depth=8, min_samples_split=12))

In [18]:
hyperparameters = {
    'max_depth': 8,
    'min_samples_split': 12
}


run_decision_tree_classifier(X_train_tfe, y_train,X_val_tfe, y_val, **hyperparameters)

Accuracy: 0.7913733983035554
Recall (Train): 0.7987955270070651
Recall (Validation): 0.7913733983035554


(                          Actual                Predicted
 772702   Closed with explanation  Closed with explanation
 641726   Closed with explanation  Closed with explanation
 799623   Closed with explanation  Closed with explanation
 1081959  Closed with explanation  Closed with explanation
 859568   Closed with explanation  Closed with explanation
 ...                          ...                      ...
 671381                    Closed  Closed with explanation
 952334                    Closed  Closed with explanation
 926664                    Closed  Closed with explanation
 899697                    Closed  Closed with explanation
 237351                    Closed  Closed with explanation
 
 [49869 rows x 2 columns],
 DecisionTreeClassifier(max_depth=8, min_samples_split=12))

In [19]:
hyperparameters = {
    'max_depth': 10,
    'min_samples_split': 20
}


run_decision_tree_classifier(X_train_tfe, y_train,X_val_tfe, y_val, **hyperparameters)

Accuracy: 0.7917945015941767
Recall (Train): 0.8053058305315857
Recall (Validation): 0.7917945015941767


(                          Actual                Predicted
 772702   Closed with explanation  Closed with explanation
 641726   Closed with explanation  Closed with explanation
 799623   Closed with explanation  Closed with explanation
 1081959  Closed with explanation  Closed with explanation
 859568   Closed with explanation  Closed with explanation
 ...                          ...                      ...
 671381                    Closed  Closed with explanation
 952334                    Closed  Closed with explanation
 926664                    Closed  Closed with explanation
 899697                    Closed  Closed with explanation
 237351                    Closed  Closed with explanation
 
 [49869 rows x 2 columns],
 DecisionTreeClassifier(max_depth=10, min_samples_split=20))

In [20]:
hyperparameters = {
    'max_depth': 7,
    'min_samples_split': 15
}


run_decision_tree_classifier(X_train_tfe, y_train,X_val_tfe, y_val, **hyperparameters)

Accuracy: 0.7913533457659067
Recall (Train): 0.7959681569959026
Recall (Validation): 0.7913533457659067


(                          Actual                Predicted
 772702   Closed with explanation  Closed with explanation
 641726   Closed with explanation  Closed with explanation
 799623   Closed with explanation  Closed with explanation
 1081959  Closed with explanation  Closed with explanation
 859568   Closed with explanation  Closed with explanation
 ...                          ...                      ...
 671381                    Closed  Closed with explanation
 952334                    Closed  Closed with explanation
 926664                    Closed  Closed with explanation
 899697                    Closed  Closed with explanation
 237351                    Closed  Closed with explanation
 
 [49869 rows x 2 columns],
 DecisionTreeClassifier(max_depth=7, min_samples_split=15))